In [ ]:
import pandas as pd
import finterstellar as fs

In [ ]:
ld = fs.LoadData()
path = './data/'

포트폴리오 선정

In [ ]:
# define universe
portfolio_dict = dict()
portfolio = {
    'World indices' : ['KOSPI 200', 'S&P 500', 'Nikkei 225', 'CSI 300']
}    # 포트폴리오를 딕셔너리 형태로 저장
p_name = 'World indices'   # 포트폴리오 집합 중 분석대상 포트폴리오의 이름 입력
p_cd = portfolio[p_name]    # 선택한 포트폴리오 내에서 처리대상 종목코드들을 지정

데이터 로딩

In [ ]:
# make new file
prices_df = ld.create_portfolio_df(path, p_name, p_cd) 
# ld 모듈의 create_portfolio_df(폴더명, 포트폴리오 이름, 종목코드) 함수를 이용해 포트폴리오 전체 시세를 한번에 수집
prices_df.tail()

In [ ]:
trd = fs.PairTrade()
# fs 라이브러리의 PairsTrade() 모듈을 호출에 trd로 지정

기준일자 세팅

In [ ]:
# 기준일자 세팅
base_date = '2018-01-01'
#base_date = pd.Timestamp.today().date() - pd.DateOffset(years=1)    # 디폴트로 1년 전 세팅
base_date = trd.check_base_date(prices_df, base_date)
# check_base_date(데이터프레임, 기준일자) - 기준일자가 공휴일인지 체크하고, 공휴일이면 다음 영업일을 반환
base_date

R2 값 비교

In [ ]:
comp_r2 = trd.compare_r2(prices_df, base_date, p_cd)
'''
    compare_r2(데이터프레임, 기준일자, [종목코드군]) 
    - [종목코드군] 내 종목 쌍 전체에 대해 회귀분석을 수행해서 R2 값 순서대로 결과 반환
'''
comp_r2.head(10)

데이터구간 샘플링

In [ ]:
pair = ['S&P 500', 'Nikkei 225']
sample = trd.sampling(prices_df, base_date, pair)
# sampling(데이터프레임, 기준일자, [종목명]) - 데이터프레임에서 지정한 종목의 기준일자 이후 데이터만 샘플링

회귀분석 수행

In [ ]:
regr = trd.regression(sample, pair)
# regression(데이터프레임, [종목쌍]) - 데이터프레임의 종목쌍에 대해 회귀분석을 수행해 기울기, y절편, R2값 반환
regr

CSI 300 지수가 x 일때 추세선에 따른 KOSPI 200 지수가 얼마이어야 하는지 구함

In [ ]:
sample = trd.expected_y(sample, regr, pair)
# expected_y(데이터프레임, 회귀분석결과, [종목쌍]) - 회귀분석결과를 집어넣고 통계적인 y종목의 기대가격을 산출
sample.head()

역치 세팅

In [ ]:
# trading strategy determinants
threshold = 0.02   # percent 값으로 오차허용 구간을 정의
sample = trd.price_analyze(sample, threshold, pair)
# price_analyze(데이터프레임, 역치, [종목쌍]) - 역치값 대비 현재 가격의 고평가/저평가 여부 구분

# 백테스팅

트레이드북 생성

In [ ]:
book = trd.create_trade_book(sample, pair)
# create_trade_book(데이터프레임, [종목쌍]) - 트레이딩북 생성
book.head()

매일매일의 매매내역 기록

In [ ]:
book = trd.tradings(sample, book, threshold, pair, short=False)
# tradings(데이터프레임, 트레이딩북, 역치, [종목코드쌍], 숏가능) - pair trade 매매전략을 이용한 백테스팅 수행

일자별 포지션 기록

In [ ]:
book = trd.position(book, pair)
# position(트레이딩북, 종목코드) - 트레이딩 전략 수행에 수반되는 포지션 판단
book.tail()

일자별 수익률 및 전체 수익률 계산

In [ ]:
fund_rtn = trd.returns(book, pair, display=True)
# returns(트레이딩북, 종목코드) - 전략의 수익률 계산

벤치마크 수익률 계산
(여기에서는 두 종목에 50%씩 분산투자하고 기간 중 계속 보유한 것으로 가정)

In [ ]:
bm_rtn = trd.benchmark_return(book, pair)
# benchmark_return(트레이딩북, 종목코드) - 벤치마크 수익률 계산

페어트레이딩 전략의 초과수익률 계산

In [ ]:
exs_rtn = trd.excess_return(fund_rtn, bm_rtn)
# excess_return(전략수익률, 벤치마크수익률) - 초과수익률 계산

In [ ]:
# 참고 : 로그수익률
fund_rtn = trd.returns_log(book, pair)
# returns_log(트레이딩북, 종목코드) - 전략의 로그수익률 계산
bm_rtn = trd.benchmark_return_log(book, pair)
# benchmark_return(트레이딩북, 종목코드) - 벤치마크 수익률 계산
exs_rtn = trd.excess_return_log(fund_rtn, bm_rtn)
# excess_return(전략수익률, 벤치마크수익률) - 초과수익률 계산

그래프 모듈 호출

In [ ]:
v = fs.Visualize()

In [ ]:
v.pair_trend_index_view(sample, threshold, pair)
# v 모듈의 pairs_trend_index_view(트레이딩북, 역치, [종목코드], (사이즈)) 함수 - pairs trade 그래프 그리기

In [ ]:
v.position_view(book, pair)
# v 모듈의 position_view(트레이딩북, 종목코드, (사이즈)) 함수 - 보유내역 그래프 그리기

주가 변동 내역

In [ ]:
v.index_view(sample, base_date, pair)